Hello, dear reviewer!
Here comes my answer to the Project 3: Data Wrangling with MongoDB.

While making my response I used the Udacity course materials mostly.
In some cases I googled for solutions and went to StackOverflow.com and official MongoDB documentation.
No other materials were used.

For my project I exported a piece of the map of Moscow, Russia.

I include my code I used for the data analysis.
Since the output is really huge, I limited the output to reasonable amounts to make it readable for you.
In the real investigation I didn't use any limits.

In [180]:
import xml.etree.cElementTree as ET
import pprint as pp
import re
import codecs
import json
import os

from pymongo import MongoClient
from os.path import join, getsize

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]

problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
doublecolon = re.compile(r'.*:.*:.*')

In [218]:
# Let's have a look at some data. Just to see what's inside the file.
# Output limited.
#
i = 50
for _, element in ET.iterparse('.\part_of_moscow.osm'):
    for tag in element.iter('tag'):
        print('{0}: {1}'.format(tag.attrib['k'], tag.attrib['v']))
        i = i - 1
    if (i < 0):
        break

UnicodeEncodeError: 'ascii' codec can't encode characters in position 0-6: ordinal not in range(128)

Immediately I got an error: 

UnicodeEncodeError: 'ascii' codec can't encode characters in position 0-6: ordinal not in range(128)

It made me think there is something wrong the the character enconding.
I solved it by using the .encode('utf-8') method.
This knowledge will be useful later as well.

In [182]:
# Let's look at  data considering Unicode:
# Output limited to 50.
#
i = 50
for _, element in ET.iterparse('.\part_of_moscow.osm'):
    for tag in element.iter('tag'):
        print('{0}: {1}'.format(tag.attrib['k'].encode('utf-8'), tag.attrib['v'].encode('utf-8')))
        i = i - 1
    if (i < 0):
        break

alt_name: Кунцево
name: Кунцево I
railway: station
alt_name: Кунцево
name: Кунцево I
railway: station
highway: traffic_signals
highway: traffic_signals
highway: traffic_signals
highway: traffic_signals
highway: traffic_signals
highway: traffic_signals
highway: traffic_signals
highway: traffic_signals
highway: traffic_signals
highway: traffic_signals
highway: traffic_signals
highway: traffic_signals
highway: traffic_signals
highway: traffic_signals
highway: traffic_signals
highway: traffic_signals
highway: traffic_signals
highway: traffic_signals
highway: traffic_signals
highway: traffic_signals
highway: traffic_signals
highway: traffic_signals
highway: traffic_signals
highway: traffic_signals
colour: orange
name: Шаболовская
name:en: Shabolovskaya
network: Московский метрополитен
operator: ГУП «Московский метрополитен»
railway: subway_entrance
colour: orange
name: Шаболовская
name:en: Shabolovskaya
network: Московский метрополитен
operator: ГУП «Московский метрополитен»
railway: subway

In [208]:
# Now let's count different tags that exist in the data.
# Output limited.
#
tags = {}
i = 100
for _, element in ET.iterparse('.\part_of_moscow.osm'):
    for tag in element.iter('tag'):
        key = tag.attrib['k'].encode('utf-8')
        if (key in tags):
            tags[key] = tags[key] + 1
        else:
            tags[key] = 1
        i = i - 1
        if (i < 0):
            break
    if (i < 0):
        break

pp.pprint(tags, width=1)

{'alt_name': 2,
 'colour': 6,
 'crossing': 2,
 'description': 2,
 'highway': 39,
 'name': 8,
 'name:cs': 2,
 'name:en': 6,
 'name:fr': 2,
 'name:it': 2,
 'name:pl': 2,
 'name:pt': 2,
 'name:sv': 2,
 'network': 6,
 'operator': 6,
 'railway': 8,
 'ref': 2,
 'wikipedia': 2}


Oh, my!!!
So many different tags! With one colon, with two colons...
Similar items such as "color" and "colour"...
Dozens of "name" tags probably with international notations...

It will take months to fully clean even this small piece of the Moscow map :(

Let me have a look at more details and then I'll make certains decisions on which cases to solve.

In [204]:
# Let's see one example of each tag...
# Output limited to 20
#
tags = {}
i = 20
for _, element in ET.iterparse('.\part_of_moscow.osm'):
    for tag in element.iter('tag'):
        key = tag.attrib['k']
        value = tag.attrib['v']
        if (not key in tags):
            tags[key] = value
            i = i - 1
    if (i < 0):
        break

for item in tags:
    print('{0}: {1}'.format(item.encode('utf-8'), tags[item].encode('utf-8')))

name:it: Avtozavodskaja
name:pt: Estação Avtosavodskaia
name:en: Shabolovskaya
name: Кунцево I
wikipedia: ru:Автозаводская (станция метро, Москва)
colour: orange
description: Калужско-Рижская линия
name:cs: Avtozavodskaja
traffic_calming: bump
was:highway: crossing
crossing: uncontrolled
name:fr: Avtozavodskaïa
alt_name: Кунцево
operator: ГУП «Московский метрополитен»
name:sv: Avtozavodskaja
railway: station
name:de: Oktober
ref: 3
name:pl: Awtozawodzkaja
highway: traffic_signals
network: Московский метрополитен


What. A. Mess...
Don't give up!!! Let's continue.

In [209]:
# Let's have a look at tags having an "address" attribute
# Output limited.
#

i = 4

for _, element in ET.iterparse('.\part_of_moscow.osm'):
    temp_dict = {}
    flag = 0
    
    for tag in element.iter('tag'):
        key = tag.attrib['k']
        temp_dict[key] = tag.attrib['v']
        if (key == 'address'):
            flag = 1
    if (flag == 1):
        print('\n===New Item===\n')
        i = i - 1
        for item in temp_dict:
            print('{0}: {1}\n'.format(item.encode('utf-8'), temp_dict[item].encode('utf-8')))
    if (i == 0):
        break


===New Item===

address: Мичуринский пр-т, д. 20, к. 1 (Олимпийская деревня)


===New Item===

amenity: police

name: Отдел по борьбе с организованной преступностью

address: Мичуринский пр-т, д. 20, к. 1 (Олимпийская деревня)


===New Item===

address: Мичуринский пр-т, д. 4, к. 2 (Олимпийская деревня-80)


===New Item===

phone: (499) 726-16-70

amenity: veterinary

name: Клиника ветринарной медицины "Лебеди"

address: Мичуринский пр-т, д. 4, к. 2 (Олимпийская деревня-80)



Ok, these lines sometimes have the whole address in one line:

StreetName St., h. HouseNumber, bld. BuildingNumber

I will parse these using the split_address_line() function.

In [214]:
# Let's look at the 'addr2' entries
# Output limited.
#
counter = 3
for _, element in ET.iterparse('.\part_of_moscow.osm'):
    temp_dict = {}
    flag = 0
    for tag in element.iter('tag'):
        key = tag.attrib['k']
        temp_dict[key] = tag.attrib['v']
        if (key.startswith('addr2')):
            flag = 1
    if (flag == 1):
        counter = counter - 1
        print('\n===New Addr2 Item===\n')
        for item in temp_dict:
            print('{0}: {1}\n'.format(item.encode('utf-8'), temp_dict[item].encode('utf-8')))
    if (counter < 0):
            break


===New Addr2 Item===

addr2:housenumber: 17


===New Addr2 Item===

addr2:street: Фруктовая улица


===New Addr2 Item===

building: apartments

addr:housenumber: 6

addr2:street: Фруктовая улица

addr2:housenumber: 17

cladr:code: 77000000000265100

building:levels: 5

addr:street: Симферопольский бульвар


===New Addr2 Item===

addr2:housenumber: 14/37



In [215]:
# Let's look at the 'addr:housenumber2' entries
# Output limited.
#
counter = 5
for _, element in ET.iterparse('.\part_of_moscow.osm'):
    temp_dict = {}
    flag = 0

    
    for tag in element.iter('tag'):
        key = tag.attrib['k']
        temp_dict[key] = tag.attrib['v']
        if (key == 'addr:housenumber2'):
            flag = 1
    if (flag == 1):
        print('\n===New Addr2 Item===\n')
        for item in temp_dict:
            print('{0}: {1}\n'.format(item.encode('utf-8'), temp_dict[item].encode('utf-8')))
        counter = counter - 1
    if (counter == 0):
            break


===New Addr2 Item===

addr:housenumber2: 12 к2 с5


===New Addr2 Item===

addr:housenumber: 12 к2

building: yes

building:levels: 5

addr:housenumber2: 12 к2 с5

addr:street: улица Доватора


===New Addr2 Item===

addr:housenumber2: 17/19


===New Addr2 Item===

addr:housenumber: 19/17

building: yes

addr:street2: проспект Вернадского

building:levels: 8

addr:housenumber2: 17/19

addr:street: улица Крупской


===New Addr2 Item===

addr:housenumber2: 7



Okay, I understood what we have here!
In Russia, when a house sits at the cross of two roads, it has the number of X/Y, where
X is the house number at Road 1 and
Y is the house number at Road 2.
OSM users sometimes add such info by making two address entries: 
'addr' - Road 1 name and house number by Road 1 and 
'addr2' - Road 2 name and house number by Road 2.
House numbers can still be in the X/Y format.
I will keep this format since it's official, but I decided to create a "secondary_address" entry for addr2 and similar entries.

The same idea is sometimes implemented as addr:street2 and addr:housenumber2.
I will take care of this case as well.

In [216]:
# Let's have a look at the "fuel" entries
# Output limited.
#
counter = 10
for _, element in ET.iterparse('.\part_of_moscow.osm'):
    temp_dict = {}
    flag = 0

    for tag in element.iter('tag'):
        key = tag.attrib['k']
#        temp_dict[key] = tag.attrib['v']
        if (key.startswith('fuel')):
            print('{0}: {1}\n'.format(key, tag.attrib['v'].encode('utf-8')))
            counter = counter - 1
    if (counter == 0):
        break


fuel:diesel: yes

fuel:octane_95: yes

fuel:octane_98: yes

fuel:diesel: yes

fuel:octane_95: yes

fuel:octane_98: yes

fuel:diesel: yes

fuel:lpg: no

fuel:octane_92: yes

fuel:octane_95: yes



Ok, it seems like this is about fuel stations.
Let me tranform this to the following structure:

fuel:octane_95: yes
fuel:octane_98: yes

will become

fuel: { octane_95: yes,
        octane_98: yes}
        
The same logic applies for "payment" entries (example data not shown):

payment:visa: yes
payment:mastercard: yes

will become

payment : { visa: yes,
            mastercard: yes}

In [217]:
# Finally let's have a look at the "building" entries
# Output limited
#
counter = 5
for _, element in ET.iterparse('.\part_of_moscow.osm'):
    temp_dict = {}
    flag = 0
    if (counter == 0):
            break    
    for tag in element.iter('tag'):
        key = tag.attrib['k']
        temp_dict[key] = tag.attrib['v']
        if (key.startswith('building')):
            parts = key.split(':')
            if (len(parts) > 1):
                if (parts[1] == 'color'):
                    flag = 1
                    #pp.pprint(element, width=1)
                    #print('{0}: {1}\n'.format(key, tag.attrib['v'].encode('utf-8')))
                    counter = counter - 1
    if (flag):
        pp.pprint(temp_dict, width=4)

{'building:color': 'grey'}
{'addr:housenumber': u'1 \u04411',
 'addr:street': u'\u041a\u0430\u043d\u0430\u0442\u0447\u0438\u043a\u043e\u0432\u0441\u043a\u0438\u0439 \u043f\u0440\u043e\u0435\u0437\u0434',
 'amenity': 'bank',
 'building': 'yes',
 'building:color': 'grey',
 'building:levels': '2',
 'operator': u'\u041e\u043a\u0435\u0430\u043d \u0411\u0430\u043d\u043a'}
{'building:color': '#E3FAFA'}
{'building': 'yes',
 'building:color': '#E3FAFA',
 'building:levels': '1'}
{'building:color': '#E3FAFA'}


Ok, we have a complicated structure for tags that start with "building".
I decided to go for the following transformation:

building: school,
building:roof: concrete,
building:roof:colour: #C7C7C7,
building:levels: 1
building:levels:underground: 1

will become

building: {type: school,
            roof: {type: concrete,
                    colour: #C7C7C7},
            levels: {amount: 1,
                    underground: 1}
           }

Other k:v pairs will remain unchanged.

Also I noticed from the outputs above that there are many international options for "name", "alt_name" and "official_name" entries. Let me go for the following transformation:

name: NameInRussian
name:en: NameInEnglish
name:de: NameInGerman

will become

name: NameInRussian
name_international: {en: NameInEnglish,
                    de: NameInGerman}
                    
Same for "alt_name" and "official_name"

That's it. There are much, MUCH more other things that could be taken care of.
But it will take me ages to go through them.
Let me focus on the fixes that I selected above and implement them.

Next come functions for the transformation process.
Note: I copied the approach for "created" and "position" from the course.

In [190]:
# Split Russian full address line into a tuple: (street, housenumber, building)
# Returns None if unsplittable (that is, something is wrong)
#
def split_address_line(address):
    parts = address.split(',')
    if (len(parts) == 3):
        street_name = parts[0].strip()
        house_number = re.search(r'[0-9]+', parts[1]).group()
        building_number = re.search(r'[0-9]+', parts[2]).group()
        return (street_name, house_number, building_number)
    else:
        return None

In [191]:
# Main function to shape map elements into JSON
#
def shape_element(element):
    node = {}
    if (element.tag == 'node') or (element.tag == 'way'):
        node['type'] = element.tag
        # Empty elements to be filled later
        temp_dict = {'created': {}, \
                    'address': {}, \
                    'secondary_address': {}, \
                    'fuel': {}, \
                    'payment': {}, \
                    'building': {}, \
                    'position': [] }
        latitude = None
        longitude = None
        
        # Handle the "created" items
        for a in element.attrib:
            if (a in CREATED):
                temp_dict['created'][a] = element.attrib[a]
            elif (a == 'lat'):
                latitude = float(element.attrib[a])
            elif (a == 'lon'):
                longitude = float(element.attrib[a])
            else:
                node[a] = element.attrib[a]
        
        if (latitude) and (longitude):
            temp_dict['position'] = [latitude, longitude]
        
        # Handle underlying tags    
        for tag in element.iter('tag'):
            key = tag.attrib['k']
            value = tag.attrib['v'].encode('utf-8')
            
            #
            # Main shaping body
            #
            if (key.startswith('addr')):
                # Handle various address items
                if (key == 'address'):
                    # Handle full address lines such as "StreetName St., h. HouseNumber, bld. BuildingNumber"
                    parts = split_address_line(value)
                    if (parts):
                        temp_dict['address']['street'] = parts[0]
                        temp_dict['address']['housenumber'] = parts[1]
                        temp_dict['address']['building'] = parts[2]
                else:
                    # Handle other address items such as "addr2:street" or "addr:street2"
                    parts = key.translate(None, '2').split(':')
                    if (len(parts) == 3):
                        # Handle addr:city:en entry
                        parts[1] = parts[1] + '_internaltional'
                    if ('2' in key):
                        # We have either "addr2:street" or "addr:street2" so we fill an secondary address
                        temp_dict['secondary_address'][parts[1]] = value
                    else:
                        # We have a primary address item
                        temp_dict['address'][parts[1]] = value
            
            elif (key.startswith('fuel')):
                # Handle fuel entries
                parts = key.split(':')
                temp_dict['fuel'][parts[1]] = value
            
            elif (key.startswith('payment')):
                # Handle payment entries
                parts = key.split(':')
                temp_dict['payment'][parts[1]] = value
            
            elif (key.startswith('building')):
                # Handle building entries
                parts = key.split(':')
                if (len(parts) == 1):
                    # Fill building type
                    if (value <> 'yes'):
                        temp_dict['building']['type'] = value
                elif (len(parts) == 2):
                    # Prepare sub-dicts for "levels", "roof" and make normal entries for others
                    if (parts[1] == 'levels') or (parts[1] == 'roof'):
                        if (parts[1] not in building):
                            temp_dict['building'][parts[1]] = {}                        
                        if (parts[1] == 'levels'):
                            temp_dict['building']['levels']['amount'] = value
                        else:
                            temp_dict['building']['roof']['type'] = value
                    else:
                        temp_dict['building'][parts[1]] = value
                else:
                    # Fill sub-dicts for "levels", "roof". Here I handle double-colon items, i.e. "building:roof:color"
                    if (parts[1] not in building):
                        temp_dict['building'][parts[1]] = {}
                    temp_dict['building'][parts[1]][parts[2]] = value
        
            elif (key.startswith('name')) or (key.startswith('alt_name')) or (key.startswith('official_name')):
                # Handle name, alt_name and official_name items
                parts = key.split(':')
                if (len(parts) > 1):
                    # We have an international entry, such as "name:en", for example
                    international_dict = parts[0] + '_international'
                    if (international_dict not in node):
                        node[international_dict] = {}
                    node[international_dict][parts[1]] = value
                else:
                    # We have normal name, alt_name and official_name items
                    node[key] = value

            else:
                # Default handler for all the rest
                # I drop all items with problematic characters and all items with double-colon
                if (re.match(problemchars, key) == None) and (re.match(doublecolon, key) == None):
                    if (key <> 'type'):
                        # There might be tags with the key of 'type' and it will interfere with the node type we set above.
                        node[key] = value
                
            # Handle nd tags for "way"
            if (element.tag == "way"):
                node['node_refs'] = []
                for tag in element.iter('nd'):
                    node['node_refs'].append(tag.attrib['ref'])
                
        # Final part. Assemble all sub-dicts together provided that they were filled
        for item in temp_dict:
            if (temp_dict[item]):
                node[item] = temp_dict[item]
        return node
    else:
        return None


In [192]:
# Main body for map processing
#
file_out = "{0}.json".format('part_of_moscow')

with codecs.open(file_out, "w") as fo:
    for _, element in ET.iterparse('.\part_of_moscow.osm'):
        el = shape_element(element)
        if el:
            fo.write(json.dumps(el) + "\n")

In [193]:
# Sizes of files I work with
#
for root, dirs, files in os.walk('.'):
    for name in files:
        if ('moscow' in name):
            print('File: {0}\t\tSize: {1} bytes'.format(name,getsize(name)))

File: part_of_moscow.json		Size: 97687326 bytes
File: part_of_moscow.osm		Size: 88206345 bytes


Ok, both files are more than 50 MB.

At this point I load my JSON file into the MongoDB instance using the following OS command:

c:\Program Files\MongoDB\Server\3.2\bin>mongoimport -d examples -c moscow --file
 "d:\Udacity\P3 Data Wrangling with MongoDB\part_of_moscow.json"
2016-04-24T13:24:17.010+0300    connected to: localhost
2016-04-24T13:24:19.985+0300    [######..................] examples.moscow
23.6 MB/93.2 MB (25.3%)
2016-04-24T13:24:22.985+0300    [############............] examples.moscow
48.6 MB/93.2 MB (52.2%)
2016-04-24T13:24:25.985+0300    [##################......] examples.moscow
72.5 MB/93.2 MB (77.8%)
2016-04-24T13:24:28.311+0300    [########################] examples.moscow
93.2 MB/93.2 MB (100.0%)
2016-04-24T13:24:28.312+0300    imported 412257 documents

c:\Program Files\MongoDB\Server\3.2\bin>

Now let's investigate data in the database with some queries.

In [194]:
# Establish a connection to the database run locally
#
client = MongoClient('mongodb://localhost:27017')

db = client.examples


In [195]:
# Number of documents in the database:
#
number_of_documents = db.moscow.find().count()

print 'Number of Moscow documents: ', number_of_documents

Number of Moscow documents:  412257


Ok, it's the same to the number of documents that were imported using mongoimport

In [196]:
# Number of 'node' and 'way' tags:
#
pipeline = [{ "$group" : { "_id" : "$type", \
                           "count" : { "$sum" : 1 } } }]

pp.pprint([doc for doc in db.moscow.aggregate(pipeline)], width=4)

[{u'_id': u'node',
  u'count': 348210},
 {u'_id': u'way',
  u'count': 64047}]


In [197]:
# Number of unique contributors.
# It's definitely a dirty way, but I didn't want to copy the nice query based on "distinct" from the submission example :)
#
pipeline = [{ "$group" : { "_id" : "Unique users", \
                            "unique_users" : { "$addToSet" : "$created.user"} } }, \
            { "$project" : { "Count" : { "$size" : "$unique_users" } } }]
    
pp.pprint([doc for doc in db.moscow.aggregate(pipeline)], width=4)


[{u'Count': 1012,
  u'_id': u'Unique users'}]


In [198]:
# Top 10 contributing users
# Note the Unicode characters in user names!
#
pipeline = [{ "$group" : { "_id" : "$created.user", \
                           "count" : { "$sum" : 1 } } }, \
            { "$sort" : { "count" : -1 } }, \
           { "$limit" : 10 }]

result = db.moscow.aggregate(pipeline)

for doc in result:
    print('User: {0}, Contributions: {1}'.format(doc['_id'].encode('utf-8'), doc['count']))


User: trolleway, Contributions: 31906
User: s-tikhomirov, Contributions: 24743
User: ad47, Contributions: 23915
User: Павел Гетманцев, Contributions: 18349
User: AMDmi3, Contributions: 16942
User: literan, Contributions: 16705
User: KekcuHa, Contributions: 13063
User: Felis Pimeja, Contributions: 12634
User: stlp, Contributions: 8886
User: SergZL, Contributions: 8573


In [199]:
# Let's see how many documents have a "secondary_address" info that I created while shaping
#
pipeline = [{ "$match" : { "secondary_address" : { "$exists" : 1 } } }, \
            { "$group" : { "_id" : "Documents having a secondary address", \
                          "count" : { "$sum" : 1 } } }]
 
pp.pprint([doc for doc in db.moscow.aggregate(pipeline)], width=4)

[{u'_id': u'Documents having a secondary address',
  u'count': 38}]


Additional ideas.

I launched the iD browser in order to see how the mapping process looks like in reality.
I noticed that users can create any tags and put any values when describing objects.

Two examples that I noticed while initially analyzing the data set:

Example 1. Some buildings have two tags: <tag k='color' .../> and <tag k='colour' .../>. 
I guess we are talking about same things here.
Obviously we have here a confusion caused by language nuances. 
The system might limit options of "colour"/"color" to just "color".

Also the "color" tag might have either literal values: grey, or hex values, such as "#E3FAFA".
Why doesn't one limit this field to just one type of data?

Example 2. In Russia houses can have "building numbers" and "house bodies". 
In fact these are same things, but there are many cases when a house has a house number, a building number and a body number: StreetName St., 17 (house number), bld. 2 (building number), bd. 1 (body number). 
This is the official notation.

I noticed in the editor that in some cases the "House Number" entry contains values such as "17 bld2".
It can be confusing when doing a search. Some people might search for "17 bld2", some people might search for "17, bld2", some for "17b2" etc.

I think map editors should enforce some policy against such fields.
For example, if a user enters letters in the "House Number" field, he might receive a warning saying that unless the official house number includes letters, user should use other fields (such as body number, building number, etc.) to specify corresponding information.

I understand that my suggestions might contradict to the XML paradigm which is about any tags, any values. However, for the sake of data uniformity and clarity, one might think of it, as sometimes some items should conform to certain rules.

In [200]:
# A couple of additional queries against the database.
#
# Let's calculate different types of buildings that we have in the database
#
pipeline = [{ "$match" : { "building.type" : { "$exists" : 1 } } }, \
            { "$group" : { "_id" : "$building.type", \
                           "count" : { "$sum" : 1 } } }, \
            { "$sort" : { "count" : -1 } }]

pp.pprint([doc for doc in db.moscow.aggregate(pipeline)], width=4)

[{u'_id': u'apartments',
  u'count': 3535},
 {u'_id': u'garages',
  u'count': 1309},
 {u'_id': u'school',
  u'count': 336},
 {u'_id': u'kindergarten',
  u'count': 214},
 {u'_id': u'commercial',
  u'count': 136},
 {u'_id': u'industrial',
  u'count': 110},
 {u'_id': u'garage',
  u'count': 91},
 {u'_id': u'office',
  u'count': 90},
 {u'_id': u'university',
  u'count': 87},
 {u'_id': u'roof',
  u'count': 77},
 {u'_id': u'service',
  u'count': 60},
 {u'_id': u'retail',
  u'count': 47},
 {u'_id': u'construction',
  u'count': 46},
 {u'_id': u'church',
  u'count': 40},
 {u'_id': u'warehouse',
  u'count': 37},
 {u'_id': u'hospital',
  u'count': 31},
 {u'_id': u'residential',
  u'count': 30},
 {u'_id': u'kiosk',
  u'count': 22},
 {u'_id': u'dormitory',
  u'count': 20},
 {u'_id': u'house',
  u'count': 20},
 {u'_id': u'public',
  u'count': 19},
 {u'_id': u'hangar',
  u'count': 11},
 {u'_id': u'chapel',
  u'count': 7},
 {u'_id': u'shop',
  u'count': 7},
 {u'_id': u'hotel',
  u'count': 6},
 {u'_id':

In [201]:
# Finally let's see how many documents have a "name_international" info that I created while shaping
# A bit more complex query than the one above...
#
pipeline = [{ "$match" : { "$or": [ \
                                   { "name_international" : { "$exists" : 1 } }, \
                                   { "alt_name_international" : { "$exists" : 1 } }, \
                                   { "official_name_international" : { "$exists" : 1 } }, \
                                   ] } }, \
            { "$group" : { "_id" : "Documents having international names", \
                          "count" : { "$sum" : 1 } } }]
    
pp.pprint([doc for doc in db.moscow.aggregate(pipeline)], width=4)

[{u'_id': u'Documents having international names',
  u'count': 1218}]


Conslusion.

Data for even a small piece of Moscow, Russia, are quite dirty.
It will require a huge amount of time and, most important, many decisions on how to handle them.
A lot of data will probably will have to be dropped and re-entered using a more unified data schema.

Now I fully understand why the task of Data Munging and Wrangling is so important and why it consumes a great deal of the Data Scientist's time.

I believe that even when one can use very flexible schemas, it might make sense to enforce some policies against the data coming into the schema.